# Visualization of the results

This notebook visualizes the results and plots samples of reconstructed pointclouds.


In [2]:
import json 
import glob
import torch
import pandas as pd
from IPython.display import display
pd.options.display.float_format = '{:,.4f}'.format
from shapesynthesis.plotting import plot_recon_2d, plot_recon_3d

In [3]:
def model_to_cate(model):
    name = model.split("_")[-1]
    return name.title()


def model_to_modelname(model):
    name_list = model.split("_")[:-1]
    name = " ".join(name_list)
    return name.title()

def model_to_modelname_rendered(model):
    name_list = model.split("_")[:-4]
    name = " ".join(name_list)
    return name.title()


def visualize_2d_model(model,num_pc=5):
    recon_pcs = torch.load(f'./results/{model}/reconstructions.pt').cpu().detach().squeeze().numpy()
    reference_pcs = torch.load(f'./results/{model}/references.pt').cpu().detach().squeeze().numpy()
    plot_recon_2d(recon_pcs=recon_pcs[:num_pc],ref_pcs=reference_pcs[:num_pc],num_pc=num_pc)

def visualize_2d_genmodel(model,num_pc=5):
    recon_pcs = torch.load(f'./results/{model}/samples.pt').cpu().detach().squeeze().numpy()
    reference_pcs = torch.load(f'./results/{model}/references.pt').cpu().detach().squeeze().numpy()
    plot_recon_2d(recon_pcs=recon_pcs[:num_pc],ref_pcs=reference_pcs[:num_pc],num_pc=num_pc)


def visualize_3d_model(model,scale=1.0,num_pc:int=5):
    recon_pcs = scale*torch.load(f'./results/{model}/reconstructions.pt').cpu().detach().squeeze().numpy()
    reference_pcs = scale*torch.load(f'./results/{model}/references.pt').cpu().detach().squeeze().numpy()
    plot_recon_3d(recon_pcs=recon_pcs,ref_pcs=reference_pcs,num_pc=num_pc)

def visualize_3d_genmodel(model,scale=1.0,num_pc:int=5,offset:int=0):
    recon_pcs = scale*torch.load(f'./results/{model}/samples.pt').cpu().detach().squeeze().numpy()
    reference_pcs = scale*torch.load(f'./results/{model}/references.pt').cpu().detach().squeeze().numpy()
    plot_recon_3d(recon_pcs=recon_pcs,ref_pcs=reference_pcs,num_pc=num_pc,offset=offset)


def visualize_2d_rendered(folder,ectsize,num_pc=5):
    recon_pcs = torch.load(f'./results/{folder}/reconstructions_{ectsize}.pt').cpu().detach().squeeze().numpy()
    reference_pcs = torch.load(f'./results/{folder}/references_{ectsize}.pt').cpu().detach().squeeze().numpy()
    plot_recon_2d(recon_pcs=recon_pcs,ref_pcs=reference_pcs,num_pc=num_pc)

In [4]:
recon_pcs = torch.load(f'./results/encoder_downsample_car/reconstructions.pt').cpu().detach().squeeze().numpy()
print(recon_pcs.shape)

(352, 2048, 3)


# Reconstruction Evaluation.

The encoder is the `encoder` and the rendered model is the point cloud
optimization rendering (a non-parametric method). The latter clearly outperforms 
all models.  

In [5]:
from pprint import pprint
import numpy as np

# Collect filenames for reconstruction.

foldernames = []
for model in ["encoder","encoder_downsample"]:
    for loss in ["", "_ect","_chamfer"]:
        for cate in ["_airplane", "_car", "_chair"]:
            foldernames.append(model+loss+cate)


filenames = []
for folder in foldernames:
    filenames.extend(glob.glob(f"./results/{folder}/**.json"))


# Both are custom
# Manual append oracle 
filenames.append("./results/oracle/results.json")
filenames.append("./results/canonicalvae/results.json")

results = []
for filename in filenames:
    with open(filename,"r") as f:
        results.extend(json.load(f))

# pd.options.display.max_rows = 999

df = pd.DataFrame(results)

df = df.drop(["fscore","normalized"],axis=1)
df["MMD-CD"] *= 1e4 
df["MMD-EMD"] *= 1e3 

df["dataset"] = df["model"].apply(model_to_cate)
df["model"] = df["model"].apply(model_to_modelname)
df = df[["MMD-CD","MMD-EMD","model","dataset"]]

# display(df)

df = df.groupby(by=["model","dataset"]).agg(["mean","std"])

df = df.unstack().swaplevel(1,2,axis=1).swaplevel(0,1,axis=1).sort_index(axis=1).reindex(columns=['Airplane', 'Chair', 'Car'], level='dataset')
df = df.replace(np.nan, 0.0)
display(df)

dataset            Airplane                         Chair                 \
                     MMD-CD        MMD-EMD         MMD-CD        MMD-EMD   
                       mean    std    mean    std    mean    std    mean   
model                                                                      
Canonicalvae         0.9794 0.0018  3.1949 0.0345  6.5633 0.0199  8.6014   
Encoder              1.0286 0.0000  1.4608 0.0000  9.5266 0.0000  8.4453   
Encoder Chamfer      1.0016 0.0000  8.8946 0.0000 10.4424 0.0000 32.4305   
Encoder Downsample   2.0406 0.0000  0.4115 0.0000 10.9620 0.0000  5.6322   
Encoder Ect          2.4132 0.0000  1.0884 0.0000 13.0640 0.0000  4.2886   
Encoder Uniform      5.5014 0.0000  2.2278 0.0000 23.9652 0.0000 14.6204   
Oracle               0.8374 0.0000  2.8256 0.0000  3.2029 0.0000  5.8276   

dataset                       Car                        
                           MMD-CD        MMD-EMD         
                      std    mean    std    mean    std  
model                                                    
Canonicalvae       0.0700  5.4384 0.0063  6.1322 0.0250  
Encoder            0.0000  6.1213 0.0000  4.1685 0.0000  
Encoder Chamfer    0.0000  5.9710 0.0000 14.9597 0.0000  
Encoder Downsample 0.0000  8.8231 0.0000  2.8345 0.0000  
Encoder Ect        0.0000  7.7497 0.0000  2.4718 0.0000  
Encoder Uniform    0.0000 18.4666 0.0000  9.3589 0.0000  
Oracle             0.0000  3.9048 0.0000  5.1768 0.0000

In [6]:
# Save the reconstruction table. 
df_list = []
df = df.round(decimals=2)
for date, new_df in df.groupby(level=[0,1],axis=1):
    # print(new_df)
    final_df = new_df.xs("mean", level=2,axis=1).astype(str) + " \pm " + new_df.xs("std", level=2,axis=1).astype(str)
    df_list.append(final_df)
    

df_latex = pd.concat(df_list,axis=1).reindex(columns=['Airplane', 'Chair', 'Car'], level='dataset')
display(df_latex)
df_latex.style.to_latex("./tables/reconstruction.tex",siunitx=True, multirow_align='c',multicol_align='c')

dataset                 Airplane                         Chair                 \
                          MMD-CD        MMD-EMD         MMD-CD        MMD-EMD   
model                                                                           
Canonicalvae        0.98 \pm 0.0  3.19 \pm 0.03  6.56 \pm 0.02   8.6 \pm 0.07   
Encoder             1.03 \pm 0.0   1.46 \pm 0.0   9.53 \pm 0.0   8.45 \pm 0.0   
Encoder Chamfer      1.0 \pm 0.0   8.89 \pm 0.0  10.44 \pm 0.0  32.43 \pm 0.0   
Encoder Downsample  2.04 \pm 0.0   0.41 \pm 0.0  10.96 \pm 0.0   5.63 \pm 0.0   
Encoder Ect         2.41 \pm 0.0   1.09 \pm 0.0  13.06 \pm 0.0   4.29 \pm 0.0   
Encoder Uniform      5.5 \pm 0.0   2.23 \pm 0.0  23.97 \pm 0.0  14.62 \pm 0.0   
Oracle              0.84 \pm 0.0   2.83 \pm 0.0    3.2 \pm 0.0   5.83 \pm 0.0   

dataset                       Car                 
                           MMD-CD        MMD-EMD  
model                                             
Canonicalvae        5.44 \pm 0.01  6.13 \pm 0.02  
Encoder              6.12 \pm 0.0   4.17 \pm 0.0  
Encoder Chamfer      5.97 \pm 0.0  14.96 \pm 0.0  
Encoder Downsample   8.82 \pm 0.0   2.83 \pm 0.0  
Encoder Ect          7.75 \pm 0.0   2.47 \pm 0.0  
Encoder Uniform     18.47 \pm 0.0   9.36 \pm 0.0  
Oracle                3.9 \pm 0.0   5.18 \pm 0.0

The rendered results.

In [7]:
fn = glob.glob("./results/rendered_ect/**/**.json")

def model_to_scale(model):
    return int(model.split("_")[-3])


def model_to_resolution(model):
    return int(model.split("_")[-4])


results = []
for filename in fn:
    with open(filename,"r") as f:
        r = json.load(f)
        if isinstance(r,list):
            results.extend(r)
        else: 
            results.extend([r])

df = pd.DataFrame(results)
df = df.drop(["fscore"],axis=1)
df["MMD-CD"] *= 1e4 
df["MMD-EMD"] *= 1e3 

df["dataset"] = df["model"].apply(model_to_cate)
df["Resolution"] = df["model"].apply(model_to_resolution)
df["Scale"] = df["model"].apply(model_to_scale)
df["model"] = df["model"].apply(model_to_modelname_rendered)

df = df.groupby(by=["model","Resolution","Scale","dataset"]).agg(["mean","std"])

df = df.unstack().swaplevel(1,2,axis=1).swaplevel(0,1,axis=1).sort_index(axis=1).sort_index(ascending=False).reindex(columns=['Airplane', 'Chair', 'Car'], level='dataset')

display(df)
# results

dataset                 Airplane                         Chair                 \
                          MMD-CD        MMD-EMD         MMD-CD        MMD-EMD   
                            mean    std    mean    std    mean    std    mean   
model  Resolution Scale                                                         
Render 128        128     1.2731 0.0034  0.2933 0.0011  3.3968 0.0066  0.6492   
                  64      0.7497 0.0010  0.1812 0.0001  2.1005 0.0025  0.4110   
                  32      0.5205 0.0005  0.1241 0.0003  1.8326 0.0024  0.3496   
       64         64      2.6529 0.0057  0.5680 0.0008  5.8310 0.0065  1.1022   
                  32      1.8752 0.0057  0.3983 0.0016  4.2919 0.0047  0.7826   
                  16      1.2229 0.0023  0.2598 0.0007  3.6202 0.0034  0.6247   
       32         32      6.2125 0.0126  1.3234 0.0021 16.9227 0.0237  3.5446   
                  16      4.1060 0.0100  0.8971 0.0024 12.5512 0.0176  2.5331   
                  8       2.7675 0.0043  0.6137 0.0024  9.0539 0.0050  1.7330   

dataset                           Car                        
                               MMD-CD        MMD-EMD         
                           std   mean    std    mean    std  
model  Resolution Scale                                      
Render 128        128   0.0014 2.9689 0.0032  0.5540 0.0016  
                  64    0.0012 2.1536 0.0032  0.4168 0.0017  
                  32    0.0008 2.0389 0.0021  0.3861 0.0012  
       64         64    0.0013 4.7559 0.0052  0.8517 0.0010  
                  32    0.0017 4.0537 0.0064  0.7031 0.0015  
                  16    0.0018 3.8012 0.0033  0.6362 0.0016  
       32         32    0.0068 9.8530 0.0081  1.9882 0.0043  
                  16    0.0028 8.5587 0.0158  1.6329 0.0017  
                  8     0.0039 7.2351 0.0082  1.3475 0.0044

In [8]:
df_list = []
df = df.round(decimals=2)
for date, new_df in df.groupby(level=[0,1],axis=1):
    # print(new_df)
    final_df = new_df.xs("mean", level=2,axis=1).astype(str) + " \pm " + new_df.xs("std", level=2,axis=1).astype(str)
    df_list.append(final_df)
    

df_latex = pd.concat(df_list,axis=1)
df_latex.style.to_latex("./tables/ectrender.tex",siunitx=True, multirow_align='c',multicol_align='c')

In [10]:
def model_to_cate_gen(model):
    name = model.split("_")[-1]
    if name == "latent":
        name = model.split("_")[-2]
    return name.title()

def model_to_modelname_gen(model):
    
    
    name_list = model.split("_")
    if  name_list[-1] =="latent":
        name_list.pop(1)
    else:
        name_list = name_list[:-1]
    name = " ".join(name_list)
    return name.title()


fn = glob.glob("./results/**/**_gen.json")

results = []
for filename in fn:
    with open(filename,"r") as f:
        results.extend(json.load(f))


df = pd.DataFrame(results)
df.rename(columns={
    'lgan_cov-CD': 'COV-CD', 
    'lgan_cov-EMD': 'COV-EMD',
    'lgan_mmd-EMD': 'MMD-EMD',
    'lgan_mmd-CD': 'MMD-CD',
    '1-NN-CD-acc' : "1-NNA-CD",
    '1-NN-EMD-acc' : "1-NNA-EMD",
    }, inplace=True)

df["1-NNA-CD"] *= 100
df["1-NNA-EMD"] *= 100
df["MMD-CD"] *= 1e3
df["MMD-EMD"] *= 1e2
df["COV-CD"] *= 1e2
df["COV-EMD"] *= 1e2

df = df[["model","MMD-EMD","MMD-CD","COV-CD","COV-EMD","1-NNA-CD","1-NNA-EMD",]]
 
df["dataset"] = df["model"].apply(model_to_cate_gen)
df["model"] = df["model"].apply(model_to_modelname_gen)
df = df.groupby(by=["model","dataset"]).agg(["mean"])

display(df.unstack().swaplevel(2,0,axis=1).sort_index(axis=1).reindex(columns=['Airplane', 'Chair', 'Car'], level='dataset').reindex(columns=["MMD-CD","MMD-EMD","COV-CD","COV-EMD","1-NNA-CD","1-NNA-EMD"], level=2))

dataset    Airplane                                             Chair          \
               mean                                              mean           
             MMD-CD MMD-EMD  COV-CD COV-EMD 1-NNA-CD 1-NNA-EMD MMD-CD MMD-EMD   
model                                                                           
Vae          0.2260  0.3199 46.6667 47.6543  86.4198   52.4691 3.2250  1.7998   
Vae Latent   0.2923  0.3907 44.2708 42.7083  85.6771   57.2917 2.5041  1.5519   

dataset                                          Car                          \
                                                mean                           
            COV-CD COV-EMD 1-NNA-CD 1-NNA-EMD MMD-CD MMD-EMD  COV-CD COV-EMD   
model                                                                          
Vae        36.4583 40.1042  69.0104   63.5417 1.3024  0.9147 34.3750 49.4792   
Vae Latent 37.4622 44.1088  64.7281   64.1994 0.9623  0.7540 34.6591 49.1477   

dataset                        
                               
           1-NNA-CD 1-NNA-EMD  
model                          
Vae         64.5833   57.8125  
Vae Latent  61.7898   61.6477